In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

    
from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
incomplete_n_samples = 2048
complete_n_samples = 4096

class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

GPU = 0
exp_counter = '7'
loss = 'chamfer'
do_training = True
load_model_conf = False
do_evaluation = True

experiment_name = exp_counter + '_' + class_type + '_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
complete_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/not_scaled', \
                             str(complete_n_samples), class_syn_id)
    
n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3]

train_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/paper_vanilla_vscan')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

if loss == 'emd':
    max_training_epochs = 100
elif loss == 'chamfer':
    max_training_epochs = 100
    
max_evaluation_epochs = max_training_epochs

Give me the class type.
sofa


In [4]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [5]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=30)
assert(len(np.unique(syn_ids)) == 1)
print '%d files containing complete point clouds were found.' % (len(cmpl_pclouds), )

3173 files containing complete point clouds were found.


In [6]:
v_scan_top_dir = osp.join(top_data_dir, 'From_Matthias/shapenet_dim32_sdf_pc', class_syn_id)

if do_training or do_evaluation:
    train_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/train_models_with_scan_id.txt')
    
    train_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, train_id_file, class_syn_id,
                                                      cmpl_pclouds, cmpl_model_names, n_threads=30,
                                                      n_samples=incomplete_n_samples)
    
    train_data, val_data = vscan.make_validation_from_train_data(train_data, 0.10)
    
if do_evaluation:
    test_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/test_models_with_scan_id.txt')
    test_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, test_id_file, class_syn_id,
                                                      cmpl_pclouds, cmpl_model_names, n_threads=30,
                                                      n_samples=incomplete_n_samples)

15600 incomplete point clouds were loaded.
3438 incomplete point clouds were loaded.


In [7]:
if load_model_conf:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)],
                    'non_linearity': tf.nn.relu
                   }

    encoder_args = {'dropout_prob': 0.5}
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 2,
                learning_rate = 0.0005,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.consistent_io = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

In [2]:
if do_training:
    training_stats = []
    training_stats.append(ae.train(train_data, conf))    
    with open(osp.join(conf.train_dir, 'train_stats.txt'), 'a') as fout:
        np.savetxt(fout, np.array(training_stats)[0])

In [9]:
if do_evaluation:    
    # Pick the epoch that minimizes the loss on the validation dataset.
    saved_epochs = np.array(read_saved_epochs(conf.train_dir))
    allowable_epochs = saved_epochs[saved_epochs <= max_evaluation_epochs]
    val_stats = eval_model(ae, conf, val_data, epochs=allowable_epochs, verbose=True)
    val_loss = np.min(val_stats[:,1])
    best_epoch = int(val_stats[np.argmin(val_stats[:,1]), 0])
    print 'Best epoch = %d.' % (best_epoch,) 
        
    ae.restore_model(conf.train_dir, best_epoch)
    top_save_dir = osp.join(conf.train_dir, 'output', 'epoch_' + str(best_epoch))
    save_dir = osp.join(top_save_dir, 'test_predictions')
    test_recon, test_loss, test_feed, test_ids, test_gt = ae.evaluate(test_data, conf)
    save_reconstructions(save_dir, test_recon, test_gt, test_feed, test_ids) # save ply files of test data.    
    train_loss = ae.evaluate(train_data, conf)[1]
    
    # Report Accuracy and Coverage of test data.
    n_examples = len(test_recon)
    pred_scores = np.zeros((n_examples, 2))
    for i in xrange(n_examples):
        gt = test_gt[i]
        pred = test_recon[i] 
        pred_scores[i, 0] = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=False)
        pred_scores[i, 1] = coverage_of_completion(gt, pred, thres=0.02, ret_dists=False)
    
    print 'Test Median-Accuracy-Coverage:', np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])
    
    save_pc_prediction_stats(osp.join(top_save_dir, 'detailed_stats.txt'), test_ids, pred_scores)
    
    with open(osp.join(top_save_dir, 'stats.txt'), 'w') as fout:
        fout.write('Best Validation Epoch = %d\n' % (best_epoch))
        fout.write('Validation loss = %f\n' % (val_loss))
        fout.write('Train loss = %f\n' % (train_loss))
        fout.write('Test loss = %f\n' % (test_loss))
        fout.write('Gen. Error (abs, per) = %f %f\n' % (abs(test_loss-train_loss),  abs(test_loss-train_loss) / train_loss ))
        fout.write('Test Median-Accuracy-Coverage = %f %f\n' % (np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])))
        fout.write('Test Median Harmonic Mean = %f' % (np.median(hmean(pred_scores, 1))))

Model restored in epoch 1.
[ 0.00129536]
Model restored in epoch 2.
[ 0.00103641]
Model restored in epoch 4.
[ 0.00088729]
Model restored in epoch 6.
[ 0.000883]
Model restored in epoch 8.
[ 0.00079982]
Model restored in epoch 10.
[ 0.0007689]
Model restored in epoch 12.
[ 0.00074846]
Model restored in epoch 14.
[ 0.00072535]
Model restored in epoch 16.
[ 0.00072094]
Model restored in epoch 18.
[ 0.00076061]
Model restored in epoch 20.
[ 0.00069646]
Model restored in epoch 22.
[ 0.0006834]
Model restored in epoch 24.
[ 0.00067364]
Model restored in epoch 26.
[ 0.00067628]
Model restored in epoch 28.
[ 0.00067416]
Model restored in epoch 30.
[ 0.00068788]
Model restored in epoch 32.
[ 0.00067058]
Model restored in epoch 34.
[ 0.00066016]
Model restored in epoch 36.
[ 0.00066993]
Model restored in epoch 38.
[ 0.00066902]
Model restored in epoch 40.
[ 0.00066088]
Model restored in epoch 42.
[ 0.00065603]
Model restored in epoch 44.
[ 0.00066154]
Model restored in epoch 46.
[ 0.00066472]
M

In [ ]:
# from geo_tool.external_code.python_plyfile.plyfile import PlyElement, PlyData
# Jet = plt.get_cmap('jet')

# def lala(color_values, pc, idx, string_tag, save_dir):
#     temp = color_values - np.max(color_values)
#     temp /= np.min(temp)
#     temp = np.squeeze(Jet(temp) * 255).astype(np.uint32)
#     pc = pc.astype(np.float32)    
#     values = np.hstack((pc, temp)).astype(np.float32)
#     vp = np.array([(v[0], v[1], v[2], v[3], v[4], v[5]) for v in values], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'),\
#                                                                              ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])
#     el = PlyElement.describe(vp, 'vertex')
#     PlyData([el], text=True).write(osp.join(save_dir, string_tag + idx ) + '.ply', )
    
# from scipy.stats import hmean

# h_measure = hmean(pred_scores, axis=1)  
# sort_ind = np.argsort(h_measure)


# n_extreme = 100
# worst_n = sort_ind[:n_extreme]
# best_n = sort_ind[-n_extreme:]



# save_dir = osp.join(top_save_dir, 'worst_test_predictions_with_color')
# create_dir(save_dir)
# n_extreme = 100
# for i in xrange(n_extreme):
#     bad = worst_n[i]
#     gt = test_gt[bad]
#     pred = test_recon[bad]   
#     feed = test_feed[bad]
#     ac_meas, ac_col = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=True)
#     cov_meas, cov_col = coverage_of_completion(gt, pred, thres=0.02, ret_dists=True)
#     ac_col[ac_col <= 0.02] = 0
#     ac_col = ac_col.astype(np.float32)
#     cov_col[cov_col <= 0.02]= 0
#     cov_col = cov_col.astype(np.float32)
#     h_meas = h_measure[bad]
#     idi = test_ids[bad]
#     print idi
#     lala(ac_col, pred, idi, str(i) + '_prediction_', save_dir)
#     lala(cov_col, gt, idi, str(i) + '_gt_', save_dir)
#     Point_Cloud(points=feed).save_as_ply(osp.join(save_dir, str(i) + '_feed_' + idi))



# save_dir = osp.join(top_save_dir, 'best_test_predictions_with_color')
# create_dir(save_dir)
# for i in xrange(n_extreme):
#     good = best_n[i]
#     gt = test_gt[good]
#     pred = test_recon[good]   
#     feed = test_feed[good]
#     ac_meas, ac_col = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=True)
#     cov_meas, cov_col = coverage_of_completion(gt, pred, thres=0.02, ret_dists=True)
#     ac_col[ac_col <= 0.02] = 0
#     ac_col = ac_col.astype(np.float32)
#     cov_col[cov_col <= 0.02]= 0
#     cov_col = cov_col.astype(np.float32)
#     h_meas = h_measure[good]
#     idi = test_ids[good]
#     print idi
#     lala(ac_col, pred, idi, str(i) + '_prediction_', save_dir)
#     lala(cov_col, gt, idi, str(i) + '_gt_', save_dir)
#     Point_Cloud(points=feed).save_as_ply(osp.join(save_dir, str(i) + '_feed_' + idi))